# PriMAT: Multi-animal tracking

In this notebook, we want to demonstrate how to train a tracking model from a few hundred frames. We will train a model that is able to track lemurs and feeding boxes with labelled images which can be downloaded [here](https://owncloud.gwdg.de/index.php/s/Mq4m9k1B74cN6ys) (-> Training Images / LemurBoxSep22 ).

In [1]:
!nvidia-smi

Thu Dec 11 10:19:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (2,767 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

In [ ]:
!sudo apt-get update -y
!sudo apt-get install -y mediainfo libmediainfo

# =========================
# Colab: installation via pip (versions épinglées) + MediaInfo/pymediainfo
# =========================
import sys, subprocess

def run(cmd):
    print(">>>", cmd)
    subprocess.check_call(cmd, shell=True)

def pip_install(*pkgs):
    cmd = [sys.executable, "-m", "pip", "install", "-U"]
    cmd += list(pkgs)
    print(">>>", " ".join(cmd))
    subprocess.check_call(cmd)

# 1) PIP et libs Python (tes versions)
pip_install("pip==23.1")
pip_install(
    "numpy==1.20.0",
    "Cython==0.29.*",
    "matplotlib==3.5.*",
    "pandas==1.4.*",
    "Pillow==9.4.*",
    "pycocotools==2.0.*",
    "seaborn==0.12.*",
    "torchmetrics==0.10.*",
    "attrs==21.4.0",
    "entrypoints==0.3",
    "iprogress==0.4",
    "joblib==1.3.1",
    "pytz==2021.3",
    "PyYAML==6.0",
    "scikit-learn==1.3.0",
    "threadpoolctl==3.1.0",
)

# 2) PyTorch CPU-only (plus simple avec versions anciennes)
run("pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu")

# =========================
# Vérifications rapides
# =========================
print("\n=== Vérifications ===")
def check_imports():
    import numpy as np
    import torch, torchvision
    import pandas as pd, matplotlib, PIL
    import pycocotools, seaborn, pymediainfo, torchmetrics, sklearn
    print("Python :", sys.version)
    print("NumPy :", np.__version__)
    print("Torch :", torch.__version__, "| Torchvision :", torchvision.__version__)
    print("Pandas:", pd.__version__, "| Matplotlib:", matplotlib.__version__)
    print("scikit-learn:", sklearn.__version__)
    print("Pillow:", PIL.__version__, "| seaborn:", seaborn.__version__)
    print("pycocotools OK:", hasattr(pycocotools, "__version__"))
    print("CUDA available:", torch.cuda.is_available())

try:
    check_imports()
except Exception as e:
    print("Erreur de vérification:", e)


In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install yacs==0.1.8
!pip install opencv-python
!pip install progress==1.6
!pip install scikit-learn==1.2.2

In [6]:
!git clone https://github.com/ecker-lab/PriMAT-tracking.git
%cd PriMAT-tracking/
!mkdir data
!mkdir models
!mkdir exp
!mkdir videos

Cloning into 'PriMAT-tracking'...
remote: Enumerating objects: 8971, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 8971 (delta 0), reused 0 (delta 0), pack-reused 8968 (from 2)
Receiving objects: 100% (8971/8971), 144.93 MiB | 14.19 MiB/s, done.
Resolving deltas: 100% (1789/1789), done.
Updating files: 100% (8972/8972), done.
/content/PriMAT-tracking


## Training

### Extract data

You have to upload the folder with the data to the colab space on the left. I uploaded it as a tar file and extracted it into the folder "data". The only important thing is that after this step you have training material in the folder data.

In [16]:

!unzip /content/PriMAT-tracking/data/LemurBoxSep22.zip >/dev/null


### Run training script

In the following cell you start the training.

1.   Élément de liste
2.   Élément de liste

Before this you have to make sure a few things:
- The data_cfg file has to point to the correct location of your data. Open it (it is in PriMAT-tracking/src/lib/cfg/lemur-box.json) and adapt the root (for me it is /content/PriMAT-tracking/data/ in colab).
- Give your experiment a name (--exp_id) so that you can find the model afterwards in exp/mot/exp_id/.

In [ ]:
%%shell
cd src
python train.py mot --exp_id colab_test\
                    --load_model ''\
                    --num_epochs 10\
                    --lr_step 5\
                    --lr '1e-4'\
                    --data_cfg '/content/PriMAT-tracking/src/lib/cfg/lemur_box.json'\
                    --store_opt\
                    --arch hrnet_32\
                    --gpus 0\
                    --batch_size 2\
                    --seed 13\
                    --reid_cls_names lemur,box\
                    --val_intervals 10\
                    --save_all
cd ..

/content/PriMAT-tracking/src/lib
training chunk_sizes: [2]
The output will be saved to  /content/PriMAT-tracking/src/lib/../../exp/mot/colab_test
Setting up data...
dataset summary
OrderedDict([('lemur_box_train', defaultdict(<class 'int'>, {0: 1361.0, 1: 1343.0}))])
start index
OrderedDict()
heads {'hm': 2, 'wh': 2, 'id': 128, 'reg': 2}
Namespace(K=50, arch='hrnet_32', batch_size=2, buffered_iou=0, cat_spec_wh=False, chunk_sizes=[2], class_names=['lemur', 'box'], clsID4GC=0, conf_thres=0.02, data_cfg='/content/PriMAT-tracking/src/lib/cfg/lemur_box.json', data_dir='../local_datasets/', dataset='jde', debug_dir='/content/PriMAT-tracking/src/lib/../../exp/mot/colab_test/debug', debug_info=False, dense_wh=False, det_thres=0.7, double_kalman=False, down_ratio=4, exp_dir='/content/PriMAT-tracking/src/lib/../../exp/mot', exp_id='colab_test', f=None, gc_cls_names='walking,sitting,standing2legs,standing4legs,NiS', gc_dim=128, gc_lbl_cnts=False, gc_loss='CrEn', gc_with_roi=False, gpus=[0], gpus

Your models will be saved in exp/mot/exp_id and end with .pth.

## Inference

### Apply to videos

If you want to have video output, you will need to activate ffmpeg.

In [ ]:
!apt-get update && apt-get install -y ffmpeg

- If you want the model you just trained, you can directly change the path to ../exp/mot/exp_id/model_last.pth (or any other model you want). Alternatively, we can use the pretrained lemur model from [here](https://owncloud.gwdg.de/index.php/s/Mq4m9k1B74cN6ys) (-> Models).
- You can upload your own videos or a validation video (e.g. Eval8.mp4) into data/Videos. Videos can be downloaded [here](https://owncloud.gwdg.de/index.php/s/Mq4m9k1B74cN6ys) (-> ValidationVideos/lemur_videos_eval/Videos/).
- If you set output_format to video, a video will be saved to output_root/output_name. If you set it to text, only the tracking output as a .txt file will be saved.

In [ ]:
!pip install motmetrics==1.2.0
!pip install lap==0.4.0
!pip install cython_bbox==0.1.3

In [ ]:
%%shell

cd src

python demo.py mot  --load_tracking_model ../models/lemur_tracking.pth\
                    --conf_thres 0.02\
                    --det_thres 0.5\
                    --new_overlap_thres 0.8\
                    --sim_thres 0.8\
                    --input_video ../data/Videos/Eval8.mp4\
                    --output_root ../videos/test/\
                    --output_name test_video\
                    --store_opt\
                    --line_thickness 2\
                    --debug_info\
                    --arch hrnet_32\
                    --output_format video\
                    --reid_cls_names "lemur,box"\
                    --proportion_iou 0.2\
                    --double_kalman


cd ..